In [100]:
from PIL import Image
import torchvision.transforms as transforms
import torch

### Importing custom modules

In [101]:
import sys

#### Importing helper

In [102]:
sys.path.append('./helper/draw')

import bounding_box

#### Importing model

In [103]:
sys.path.append('./model')

import faster_r_cnn

### Testing model

In [104]:
model = faster_r_cnn.FasterRCNNResNet101()
model.load_state_dict(torch.load('../model/faster_rcnn_resnet101_v2.pth'))

C:\Users\Brandon\AppData\Local\Temp\ipykernel_21784\414414127.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../model/faster_rcnn_resn

<All keys matched successfully>

In [105]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

FasterRCNNResNet101(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=1e-05)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=1e-05)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=1e-05)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [106]:
transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.ToTensor()
])

In [107]:
model.eval()

image = Image.open('../d/test/image/062092.jpg').convert('RGB')
image_tensor = transform(image)

image_input = list(image_tensor.to(device).unsqueeze(0))

with torch.no_grad():
    pred = model(image_input)[0]

score_threshold = 0.5
scores = pred['scores']
mask = scores >= score_threshold

filtered_boxes = pred['boxes'][mask]
filtered_labels = pred['labels'][mask]
filtered_scores = pred['scores'][mask]

In [108]:
image_with_boxes = bounding_box.draw_all_item_boxes(image, filtered_boxes, filtered_labels)
image_with_boxes.show()

print(filtered_labels)

tensor([8, 1, 8], device='cuda:0')
